#### Testing Python Script Execution

In [9]:
def fibo(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a
        

In [10]:
fibo(10)

55

### Connecting to JUPYTER NOTEBOOKS to run SPARK programs:

##### To start Jupyter notebook from 'enthought canopy command prompt', enter below commands:
##### enter spark directory from the canopy command prompt

cd C:/spark
(User) C:\spark>SET PYSPARK_DRIVER_PYTHON=jupyter <br>
(User) C:\spark>SET PYSPARK_DRIVER_PYTHON_OPTS=notebook <br>
(User) C:\spark>pyspark

### 1. Apache Spark Program to count the number of words in a file on local system
#### * No need to create a spark context here, as we have a sc already created from interactive python shell launched using pyspark

In [13]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("Local").setAppName("WordCount")
# spark context not required here
#sc = SparkContext(conf = conf)

lines = sc.textFile("file:///spark/README.md")
wordsCount = lines.count()

print ("Count of words:", wordsCount)

Count of words: 103


### 2. Resilient Distributed Dataset
#### Resilient collection of elements that can be operated upon in parallel <br>
#### Parallelized Collection

In [5]:
data = ['a','b','c','d','d','a','b']
rdd = sc.parallelize(data)

In [44]:
rddCount = rdd.count()
print (rddCount)

7


One important parameter for parallel collections is the number of partitions to cut the dataset into. 
Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster. Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parallelize (e.g. sc.parallelize(data, 10))

In [48]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data, 2)

In [50]:
distDataSum = distData.reduce(lambda a,b:a+b)
print ('distDataSum :', distDataSum)

distDataSum : 15


#### Transformation on an RDD returns an RDD

In [36]:
rddGroupByKey = rdd.groupByKey()

In [41]:
print (rddGroupByKey)

PythonRDD[46] at RDD at PythonRDD.scala:48


#### Action on an RDD returns python default dictionary

In [62]:
rddCountByKey = rdd.countByKey()

In [63]:
print (rddCountByKey)

defaultdict(<class 'int'>, {'a': 2, 'b': 2, 'd': 2, 'c': 1})


In [64]:
for i, j in rddCountByKey.items():
    print (i,j)

a 2
b 2
d 2
c 1


### External Datasets

Text file RDDs can be created using SparkContext’s textFile method.

All of Spark’s file-based input methods, including textFile, support running on directories, compressed files, and wildcards as well. 
For example, you can use textFile("/my/directory"), textFile("/my/directory/*.txt"), and textFile("/my/directory/*.gz").



The textFile method also takes an optional second argument for controlling the number of partitions of the file. By default, Spark creates one partition for each block of the file (blocks being 128MB by default in HDFS), but you can also ask for a higher number of partitions by passing a larger value. Note that you cannot have fewer partitions than blocks.

In [74]:
distFile = sc.textFile("C:/SparkCourse/book.txt")

In [52]:
distFileDir = sc.textFile("C:/SparkCourse/*.py")

##### SparkContext.wholeTextFiles 
lets you read a directory containing multiple small text files, and returns each of them as (filename, content) pairs. This is in contrast with textFile, which would return one record per line in each file.

In [67]:
distWholeTextFile = sc.wholeTextFiles("C:/SparkCourse")

RDD.saveAsPickleFile and SparkContext.pickleFile support saving an RDD in a simple format consisting of pickled Python objects. Batching is used on pickle serialization, with default batch size 10

In [76]:
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd.saveAsSequenceFile("path/to/file")
sorted(sc.sequenceFile("path/to/file").collect())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsSequenceFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:96)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:10